In [4]:
import sys
from pyspark.sql import SparkSession, functions as F
import pandas


In [173]:
spark = (
    SparkSession
    .builder
    # .master('dev-java.home.lan')
    .master('local[4]')
    .appName('mySpark')
    .getOrCreate()
)


In [184]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType, BooleanType

sm = StructType(fields = [
    StructField("lc", StringType()),
    StructField("ln", StringType()),
    StructField("lns", StringType()),
    StructField("activeFlag", StringType()),
    StructField("sd", DateType()),
    StructField("ed", DateType()),
    #StructField("cd", StringType()),
    #StructField("ca", StringType()),
    #StructField("snapshot_", StringType()),
    #StructField("cv", TimestampType()),
    #StructField("cl", IntegerType())
])
df = spark.read.csv("sbrf.csv", schema=sm, sep=',')
#df.printSchema()
df.show(25)


+----+------+-------+----------+----------+----------+
|  lc|    ln|    lns|activeFlag|        sd|        ed|
+----+------+-------+----------+----------+----------+
|lc-5|ln-5-2|lns-5-2|         1|2020-01-03|2079-06-06|
|lc-1|ln-1-1|lns-1-1|         1|2020-12-25|2021-01-01|
|lc-1|ln-1-3|lns-1-3|         1|2021-02-01|2079-06-06|
|lc-1|ln-1-2|lns-1-2|         1|2021-01-02|2021-01-31|
|lc-2|ln-2-3|lns-2-3|         0|2024-12-05|2099-01-01|
|lc-2|ln-2-1|lns-2-1|         0|2020-12-02|2024-12-01|
|lc-2|ln-2-2|lns-2-2|         0|2024-12-02|2024-12-04|
|lc-3|ln-3-1|lns-3-1|         0|2024-12-02|2019-01-02|
|lc-4|ln-4-1|lns-4-1|         1|2020-01-01|2021-01-02|
|lc-5|ln-5-1|lns-5-1|         1|2020-01-01|2021-01-02|
+----+------+-------+----------+----------+----------+



In [189]:
df = df.withColumn("af", F.col("activeFlag").cast(BooleanType()))
#df_ = df.withColumn("dtn", F.to_timestamp("dt1", "dd-MM-yyyy HH:mm:ss"))
# df_.printSchema()
df.orderBy(["lc","ln"]).show(25, truncate=False)


+----+------+-------+----------+----------+----------+-----+
|lc  |ln    |lns    |activeFlag|sd        |ed        |af   |
+----+------+-------+----------+----------+----------+-----+
|lc-1|ln-1-1|lns-1-1|1         |2020-12-25|2021-01-01|true |
|lc-1|ln-1-2|lns-1-2|1         |2021-01-02|2021-01-31|true |
|lc-1|ln-1-3|lns-1-3|1         |2021-02-01|2079-06-06|true |
|lc-2|ln-2-1|lns-2-1|0         |2020-12-02|2024-12-01|false|
|lc-2|ln-2-2|lns-2-2|0         |2024-12-02|2024-12-04|false|
|lc-2|ln-2-3|lns-2-3|0         |2024-12-05|2099-01-01|false|
|lc-3|ln-3-1|lns-3-1|0         |2024-12-02|2019-01-02|false|
|lc-4|ln-4-1|lns-4-1|1         |2020-01-01|2021-01-02|true |
|lc-5|ln-5-1|lns-5-1|1         |2020-01-01|2021-01-02|true |
|lc-5|ln-5-2|lns-5-2|1         |2020-01-03|2079-06-06|true |
+----+------+-------+----------+----------+----------+-----+



In [190]:
from pyspark.sql import Window
w = Window.partitionBy("lc").orderBy("sd")
#w = Window.orderBy("ln")
df1 = df.withColumn("nv", F.lead("sd").over(w))
df.withColumn("nv", F.lead("sd").over(w)).where('1=1 and sd <= ed').show()


+----+------+-------+----------+----------+----------+-----+----------+
|  lc|    ln|    lns|activeFlag|        sd|        ed|   af|        nv|
+----+------+-------+----------+----------+----------+-----+----------+
|lc-1|ln-1-1|lns-1-1|         1|2020-12-25|2021-01-01| true|2021-01-02|
|lc-1|ln-1-2|lns-1-2|         1|2021-01-02|2021-01-31| true|2021-02-01|
|lc-1|ln-1-3|lns-1-3|         1|2021-02-01|2079-06-06| true|      NULL|
|lc-2|ln-2-1|lns-2-1|         0|2020-12-02|2024-12-01|false|2024-12-02|
|lc-2|ln-2-2|lns-2-2|         0|2024-12-02|2024-12-04|false|2024-12-05|
|lc-2|ln-2-3|lns-2-3|         0|2024-12-05|2099-01-01|false|      NULL|
|lc-4|ln-4-1|lns-4-1|         1|2020-01-01|2021-01-02| true|      NULL|
|lc-5|ln-5-1|lns-5-1|         1|2020-01-01|2021-01-02| true|2020-01-03|
|lc-5|ln-5-2|lns-5-2|         1|2020-01-03|2079-06-06| true|      NULL|
+----+------+-------+----------+----------+----------+-----+----------+



In [181]:
df1.show(truncate=False)

+----+------+-------+----------+----------+----------+-----+----------+
|lc  |ln    |lns    |activeFlag|sd        |ed        |af   |nv        |
+----+------+-------+----------+----------+----------+-----+----------+
|lc-1|ln-1  |lns-1  |1         |2020-12-25|2021-01-01|true |2021-01-02|
|lc-1|ln-1-2|lns-1  |1         |2021-01-02|2021-01-31|true |2021-02-01|
|lc-1|ln-1-3|lns-1  |1         |2021-02-01|2099-01-01|true |NULL      |
|lc-2|ln-2  |lns-2  |0         |2020-12-02|2024-12-01|false|2024-12-02|
|lc-2|ln-2-1|lns-2  |0         |2024-12-02|2099-01-01|false|2024-12-02|
|lc-2|ln-2-2|lns-2  |0         |2024-12-02|2099-01-01|false|NULL      |
|lc-3|ln-3  |lns-3  |0         |2024-12-02|2019-01-02|false|NULL      |
|lc-4|ln-4  |lns-4-1|1         |2020-01-01|2021-01-02|true |NULL      |
|lc-5|ln-4  |lns-4-1|1         |2020-01-01|2021-01-02|true |2020-01-03|
|lc-5|ln-4  |lns-4-1|1         |2020-01-03|2079-06-06|true |NULL      |
+----+------+-------+----------+----------+----------+-----+----

In [183]:
df.createOrReplaceTempView("view1")
spark.sql("select *, lead(sd) over(partition by lc order by sd) as nv from view1 where 1=1").show()
dft1 = spark.sql("select *, lead(sd) over(partition by lc order by sd) as nv from view1 where 1=1 and sd <= ed and lc is not null")
dft1.show(25)
dft1.createOrReplaceTempView("view2")
#dft2 = spark.sql("select * from view2")
#dft2.show(25)


+----+------+-------+----------+----------+----------+-----+----------+
|  lc|    ln|    lns|activeFlag|        sd|        ed|   af|        nv|
+----+------+-------+----------+----------+----------+-----+----------+
|lc-1|  ln-1|  lns-1|         1|2020-12-25|2021-01-01| true|2021-01-02|
|lc-1|ln-1-2|  lns-1|         1|2021-01-02|2021-01-31| true|2021-02-01|
|lc-1|ln-1-3|  lns-1|         1|2021-02-01|2099-01-01| true|      NULL|
|lc-2|  ln-2|  lns-2|         0|2020-12-02|2024-12-01|false|2024-12-02|
|lc-2|ln-2-1|  lns-2|         0|2024-12-02|2099-01-01|false|2024-12-02|
|lc-2|ln-2-2|  lns-2|         0|2024-12-02|2099-01-01|false|      NULL|
|lc-3|  ln-3|  lns-3|         0|2024-12-02|2019-01-02|false|      NULL|
|lc-4|  ln-4|lns-4-1|         1|2020-01-01|2021-01-02| true|      NULL|
|lc-5|  ln-4|lns-4-1|         1|2020-01-01|2021-01-02| true|2020-01-03|
|lc-5|  ln-4|lns-4-1|         1|2020-01-03|2079-06-06| true|      NULL|
+----+------+-------+----------+----------+----------+-----+----

In [182]:
#df1.show()
df1.createOrReplaceTempView("view3")
#df1.show()
dft3 = spark.sql("select *, case af when true then false else true end as af1 from view3 where nv is null and ed <> to_date('2079-06-06')")
dft3.show()

e = spark.sql("with pre_r as (select lc,ln,lns,activeFlag,sd,ed,CAST(activeFlag as BOOLEAN) as af,lead(lc) over(partition by lc order by sd) as nv from view3 where 1=1 and sd <= ed and lc is not null) select * from pre_r")
df4 = e.select('*')
df4.show()


+----+------+-------+----------+----------+----------+-----+----+-----+
|  lc|    ln|    lns|activeFlag|        sd|        ed|   af|  nv|  af1|
+----+------+-------+----------+----------+----------+-----+----+-----+
|lc-1|ln-1-3|  lns-1|         1|2021-02-01|2099-01-01| true|NULL|false|
|lc-2|ln-2-2|  lns-2|         0|2024-12-02|2099-01-01|false|NULL| true|
|lc-3|  ln-3|  lns-3|         0|2024-12-02|2019-01-02|false|NULL| true|
|lc-4|  ln-4|lns-4-1|         1|2020-01-01|2021-01-02| true|NULL|false|
+----+------+-------+----------+----------+----------+-----+----+-----+

+----+------+-------+----------+----------+----------+-----+----+
|  lc|    ln|    lns|activeFlag|        sd|        ed|   af|  nv|
+----+------+-------+----------+----------+----------+-----+----+
|lc-1|  ln-1|  lns-1|         1|2020-12-25|2021-01-01| true|lc-1|
|lc-1|ln-1-2|  lns-1|         1|2021-01-02|2021-01-31| true|lc-1|
|lc-1|ln-1-3|  lns-1|         1|2021-02-01|2099-01-01| true|NULL|
|lc-2|  ln-2|  lns-2|      

In [169]:
df4.createOrReplaceTempView("view4")
df4.select("*").where("nv is NULL and ed <> to_date('2079-06-06')").show()

+----+------+-------+----------+----------+----------+-----+----+
|  lc|    ln|    lns|activeFlag|        sd|        ed|   af|  nv|
+----+------+-------+----------+----------+----------+-----+----+
|lc-1|ln-1-3|  lns-1|         1|2021-02-01|2099-01-01| true|NULL|
|lc-2|ln-2-2|  lns-2|         0|2024-12-02|2099-01-01|false|NULL|
|lc-4|  ln-4|lns-4-1|         1|2020-01-01|2021-01-02| true|NULL|
+----+------+-------+----------+----------+----------+-----+----+

